# Final Project

* DS 5001: Exploratory Text Analytics
* Rehan Merchant rm2bt

In [1]:
import pandas as pd
import numpy as np
from glob import glob #read directory system to import files
import re
import nltk

%matplotlib inline

In [2]:
OHCO = ['book_id', 'chap_num', 'para_num', 'sent_num', 'token_num']

import file

In [29]:
epub_file = "pg42671.txt"
#epub_file = 'pg121.txt'
epub = open(epub_file, 'r', encoding='utf-8-sig').readlines()
df = pd.DataFrame(epub, columns=['line_str'])
df.index.name = 'line_num'
df.line_str = df.line_str.str.strip()

extract book id

In [30]:
book_id = int(epub_file.split('-')[-1].split('.')[0].replace('pg',''))
print("BOOK ID", book_id)

BOOK ID 42671


In [31]:
df['book_id'] = book_id

extract title

In [32]:
title = df.loc[0].line_str.replace('The Project Gutenberg eBook, ', '')
df['title'] = title


In [33]:
print(title)

Pride and Prejudice, by Jane Austen, Edited


In [34]:
df

,line_str,book_id,title
line_num,,,
0,"The Project Gutenberg eBook, Pride and Prejudi...",42671,"Pride and Prejudice, by Jane Austen, Edited"
1,by R. W. (Robert William) Chapman,42671,"Pride and Prejudice, by Jane Austen, Edited"
2,,42671,"Pride and Prejudice, by Jane Austen, Edited"
3,,42671,"Pride and Prejudice, by Jane Austen, Edited"
4,This eBook is for the use of anyone anywhere a...,42671,"Pride and Prejudice, by Jane Austen, Edited"
...,...,...,...
13708,,42671,"Pride and Prejudice, by Jane Austen, Edited"
13709,This Web site includes information about Proje...,42671,"Pride and Prejudice, by Jane Austen, Edited"
13710,including how to make donations to the Project...,42671,"Pride and Prejudice, by Jane Austen, Edited"


Remove Gutenberg's front and back matter

In [35]:
a = df.line_str.str.match(r"\*\*\**START OF THE PROJECT ")
b = df.line_str.str.match(r"\*\*\**END OF THE PROJECT ")

In [36]:
an = df.loc[a].index[0]
bn = df.loc[b].index[0]

In [12]:
df = df.loc[an + 1 : bn - 2]

In [13]:
df

,line_str,book_id,title
line_num,,,
26,,42671,"Pride and Prejudice, by Jane Austen, Edited"
27,,42671,"Pride and Prejudice, by Jane Austen, Edited"
28,"E-text prepared by Greg Weeks, Jon Hurst, Mary...",42671,"Pride and Prejudice, by Jane Austen, Edited"
29,Distributed Proofreading Team (http://www.pgdp...,42671,"Pride and Prejudice, by Jane Austen, Edited"
30,generously made available by Internet Archive ...,42671,"Pride and Prejudice, by Jane Austen, Edited"
...,...,...,...
13350,Spelling and hyphen changes have been made so ...,42671,"Pride and Prejudice, by Jane Austen, Edited"
13351,within the book. Any other inconsistencies wit...,42671,"Pride and Prejudice, by Jane Austen, Edited"
13352,been left as printed.,42671,"Pride and Prejudice, by Jane Austen, Edited"


chunk by chapter

In [14]:
chap_lines = df.line_str.str.match(r"(chapter|letter)\s", case=False)

In [15]:
df.loc[chap_lines]

,line_str,book_id,title
line_num,,,
97,CHAPTER I.,42671,"Pride and Prejudice, by Jane Austen, Edited"
217,CHAPTER II.,42671,"Pride and Prejudice, by Jane Austen, Edited"
327,CHAPTER III.,42671,"Pride and Prejudice, by Jane Austen, Edited"
498,CHAPTER IV.,42671,"Pride and Prejudice, by Jane Austen, Edited"
612,CHAPTER V.,42671,"Pride and Prejudice, by Jane Austen, Edited"
...,...,...,...
12315,CHAPTER XV.,42671,"Pride and Prejudice, by Jane Austen, Edited"
12493,CHAPTER XVI.,42671,"Pride and Prejudice, by Jane Austen, Edited"
12761,CHAPTER XVII.,42671,"Pride and Prejudice, by Jane Austen, Edited"


Assign numbers to chapters

In [16]:
chap_nums = [i+1 for i in range(df.loc[chap_lines].shape[0])]

In [17]:
df.loc[chap_lines, 'chap_num'] = chap_nums

/Users/rehanmerchant/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/rehanmerchant/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [18]:
df.chap_num = df.chap_num.ffill()

/Users/rehanmerchant/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


clean up

In [19]:
df = df.loc[~df.chap_num.isna()] # Remove chapter heading lines
df = df.loc[~chap_lines] # Remove everything before Chapter 1
df.chap_num = df.chap_num.astype('int') # Convert chap_num from float to int

In [20]:
df.sample(10)

,line_str,book_id,title,chap_num
line_num,,,,
9056,"hills behind the house, and of the beautiful o...",42671,"Pride and Prejudice, by Jane Austen, Edited",46
10490,"through her mother's hands, Lydia's expences h...",42671,"Pride and Prejudice, by Jane Austen, Edited",54
5550,"were got into the garden, and here is nothing ...",42671,"Pride and Prejudice, by Jane Austen, Edited",29
9683,,42671,"Pride and Prejudice, by Jane Austen, Edited",50
9681,,42671,"Pride and Prejudice, by Jane Austen, Edited",50
11636,"""Mrs. Annesley is with her. The others have be...",42671,"Pride and Prejudice, by Jane Austen, Edited",59
10450,,42671,"Pride and Prejudice, by Jane Austen, Edited",54
8259,"whether Pemberley were not a very fine place, ...",42671,"Pride and Prejudice, by Jane Austen, Edited",43
2665,"Wickham, and give him an invitation also, if t...",42671,"Pride and Prejudice, by Jane Austen, Edited",15


group lines by chapter num

In [21]:
dfc = df.groupby(OHCO[1:2]).line_str.apply(lambda x: '\n'.join(x)).to_frame() # Make big string

In [22]:
dfc.head(10)

,line_str
chap_num,
1,"\n\nIt is a truth universally acknowledged, th..."
2,\n\nMr. Bennet was among the earliest of those...
3,"\n\nNot all that Mrs. Bennet, however, with th..."
4,"\n\nWhen Jane and Elizabeth were alone, the fo..."
5,\n\nWithin a short walk of Longbourn lived a f...
6,\n\nThe ladies of Longbourn soon waited on tho...
7,\n\nMr. Bennet's property consisted almost ent...
8,\n\nAt five o'clock the two ladies retired to ...
9,\n\nElizabeth passed the chief of the night in...


split into paragraphs

In [23]:
dfp = dfc['line_str'].str.split(r'\n\n+', expand=True).stack()\
    .to_frame().rename(columns={0:'para_str'})

In [24]:
dfp.head(10)

para_str
chap_num                                                     
1        0                                                   
         1  It is a truth universally acknowledged, that a...
         2  However little known the feelings or views of ...
         3  "My dear Mr. Bennet," said his lady to him one...
         4                Mr. Bennet replied that he had not.
         5  "But it is," returned she; "for Mrs. Long has ...
         6                         Mr. Bennet made no answer.
         7  "Do not you want to know who has taken it?" cr...
         8  "_You_ want to tell me, and I have no objectio...
         9                        This was invitation enough.

In [25]:
dfp.index.names = OHCO[1:3]

In [26]:
dfp.head(10)

para_str
chap_num para_num                                                   
1        0                                                          
         1         It is a truth universally acknowledged, that a...
         2         However little known the feelings or views of ...
         3         "My dear Mr. Bennet," said his lady to him one...
         4                       Mr. Bennet replied that he had not.
         5         "But it is," returned she; "for Mrs. Long has ...
         6                                Mr. Bennet made no answer.
         7         "Do not you want to know who has taken it?" cr...
         8         "_You_ want to tell me, and I have no objectio...
         9                               This was invitation enough.

In [27]:
dfp['para_str'] = dfp['para_str'].str.replace(r'\n', ' ').str.strip()
dfp = dfp[~dfp['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs

In [28]:
dfp.head(10)

para_str
chap_num para_num                                                   
1        1         It is a truth universally acknowledged, that a...
         2         However little known the feelings or views of ...
         3         "My dear Mr. Bennet," said his lady to him one...
         4                       Mr. Bennet replied that he had not.
         5         "But it is," returned she; "for Mrs. Long has ...
         6                                Mr. Bennet made no answer.
         7         "Do not you want to know who has taken it?" cr...
         8         "_You_ want to tell me, and I have no objectio...
         9                               This was invitation enough.
         10        "Why, my dear, you must know, Mrs. Long says t...

In [29]:
DOC = dfp

In [30]:
DOC['book_id'] = book_id

In [31]:
DOC = DOC.reset_index().set_index(OHCO[:3])

In [32]:
DOC

para_str
book_id chap_num para_num                                                   
42671   1        1         It is a truth universally acknowledged, that a...
                 2         However little known the feelings or views of ...
                 3         "My dear Mr. Bennet," said his lady to him one...
                 4                       Mr. Bennet replied that he had not.
                 5         "But it is," returned she; "for Mrs. Long has ...
...                                                                      ...
        67       14        Lady Catherine was extremely indignant on the ...
                 15        With the Gardiners, they were always on the mo...
                 16                        *       *       *       *       *
                 17                                      Transcriber's note:
                 18        Spelling and hyphen changes have been made so ...

[2085 rows x 1 columns]

Tokenize

In [33]:
def tokenize(doc_df, remove_pos_tuple=False, OHCO=OHCO):
    
    # Paragraphs to Sentences
    df = doc_df.para_str\
        .apply(lambda x: pd.Series(nltk.sent_tokenize(x)))\
        .stack()\
        .to_frame()\
        .rename(columns={0:'sent_str'})
    
    # Sentences to Tokens
    # .apply(lambda x: pd.Series(nltk.pos_tag(nltk.word_tokenize(x))))\
    df = df.sent_str\
        .apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
        .stack()\
        .to_frame()\
        .rename(columns={0:'pos_tuple'})
    
    # Grab info from tuple
    df['pos'] = df.pos_tuple.apply(lambda x: x[1])
    df['token_str'] = df.pos_tuple.apply(lambda x: x[0])
    if remove_pos_tuple:
        df = df.drop('pos_tuple', 1)
    
    # Add index
    df.index.names = OHCO
    
    return df

In [34]:
TOKEN = tokenize(DOC)

In [35]:
TOKEN

pos_tuple  pos  \
book_id chap_num para_num sent_num token_num                           
42671   1        1        0        0                  (It, PRP)  PRP   
                                   1                  (is, VBZ)  VBZ   
                                   2                    (a, DT)   DT   
                                   3                (truth, NN)   NN   
                                   4          (universally, RB)   RB   
...                                                         ...  ...   
        67       18       1        6                (have, VBP)  VBP   
                                   7                (been, VBN)  VBN   
                                   8                (left, VBN)  VBN   
                                   9                   (as, IN)   IN   
                                   10            (printed., NN)   NN   

                                                token_str  
book_id chap_num para_num sent_num token_num               
42671   1        1        0        0                   It  
                                   1                   is  
                                   2                    a  
                                   3                truth  
                                   4          universally  
...                                                   ...  
        67       18       1        6                 have  
                                   7                 been  
                                   8                 left  
                                   9                   as  
                                   10            printed.  

[121661 rows x 3 columns]

In [50]:
my_lib = []
author = 'austen'
my_lib.append((book_id, title,epub_file,author,title))
LIB = pd.DataFrame(my_lib, columns=['book_id', 'book_title','book_file','author','title']).set_index('book_id')

In [51]:
LIB

,book_title,book_file,author,title
book_id,,,,
42671,"Pride and Prejudice, by Jane Austen, Edited",epubs/pg42671.txt,austen,"Pride and Prejudice, by Jane Austen, Edited"


Reduce

In [52]:
TOKEN['term_str'] = TOKEN['token_str'].str.lower().str.replace('[\W_]', '')

VOCAB = TOKEN.term_str.value_counts().to_frame().rename(columns={'index':'term_str', 'term_str':'n'})\
    .sort_index().reset_index().rename(columns={'index':'term_str'})
VOCAB.index.name = 'term_id'

VOCAB['num'] = VOCAB.term_str.str.match("\d+").astype('int')

In [53]:
VOCAB

,term_str,n,num
term_id,,,
0,,42,0
1,15th,1,1
2,1813,2,1
3,18th,1,1
4,2,1,1
...,...,...,...
6937,yourselfand,1,0
6938,yourselves,2,0
6939,youth,9,0


annotate vocab

In [54]:
sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
sw = sw.reset_index().set_index('term_str')
sw.columns = ['dummy']
sw.dummy = 1

In [55]:
VOCAB['stop'] = VOCAB.term_str.map(sw.dummy)
VOCAB['stop'] = VOCAB['stop'].fillna(0).astype('int')

In [56]:
VOCAB[VOCAB.stop == 1].sample(10)

,term_str,n,num,stop
term_id,,,,
6530,up,121,0,1
6160,their,439,0,1
3026,his,1268,0,1
3471,it,1503,0,1
5576,she,1693,0,1
4335,ours,2,0,1
758,both,69,0,1
6849,won,4,0,1
4293,only,214,0,1


porter stems

In [57]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
VOCAB['p_stem'] = VOCAB.term_str.apply(stemmer.stem)

VOCAB.sample(10)

,term_str,n,num,stop,p_stem
term_id,,,,,
2055,elizabeths,37,0,0,elizabeth
6585,venture,5,0,0,ventur
4968,rapturous,2,0,0,raptur
3088,hurried,10,0,0,hurri
2253,exciting,1,0,0,excit
5709,smiled,18,0,0,smile
2753,goodnatured,4,0,0,goodnatur
3647,legacy,1,0,0,legaci
3812,manage,3,0,0,manag


In [58]:
TOKEN

pos_tuple  pos  \
book_id chap_num para_num sent_num token_num                           
42671   1        1        0        0                  (It, PRP)  PRP   
                                   1                  (is, VBZ)  VBZ   
                                   2                    (a, DT)   DT   
                                   3                (truth, NN)   NN   
                                   4          (universally, RB)   RB   
...                                                         ...  ...   
        67       18       1        6                (have, VBP)  VBP   
                                   7                (been, VBN)  VBN   
                                   8                (left, VBN)  VBN   
                                   9                   (as, IN)   IN   
                                   10            (printed., NN)   NN   

                                                token_str     term_str  
book_id chap_num para_num sent_num token_num                            
42671   1        1        0        0                   It           it  
                                   1                   is           is  
                                   2                    a            a  
                                   3                truth        truth  
                                   4          universally  universally  
...                                                   ...          ...  
        67       18       1        6                 have         have  
                                   7                 been         been  
                                   8                 left         left  
                                   9                   as           as  
                                   10            printed.      printed  

[121661 rows x 4 columns]

In [59]:
token1 = TOKEN
pos_max = token1.groupby(['term_str',"pos"]).count().sort_values("token_str", ascending = False).groupby(level=0).head(1)\
    .reset_index().set_index('term_str')
pos_max.sort_index().tail(200)
VOCAB['pos_max'] = VOCAB.term_str.map(pos_max.pos)
VOCAB

,term_str,n,num,stop,p_stem,pos_max
term_id,,,,,,
0,,42,0,0,,NNP
1,15th,1,1,0,15th,CD
2,1813,2,1,0,1813,CD
3,18th,1,1,0,18th,CD
4,2,1,1,0,2,CD
...,...,...,...,...,...,...
6937,yourselfand,1,0,0,yourselfand,NN
6938,yourselves,2,0,1,yourselv,NN
6939,youth,9,0,0,youth,NN


save to csv

In [60]:
TOKEN.to_csv('TOKEN.csv')
DOC.to_csv('DOC.csv')
LIB.to_csv('LIB.csv')
VOCAB.to_csv('VOCAB.csv')